In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install openpyxl==3.0.0    # upgrade from version 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import random
import multiprocessing as mp
# import dill
import progressbar
import datetime

%load_ext autoreload
%autoreload 2

In [3]:
# !pip uninstall geopandas
!pip install pandas fiona shapely pyproj rtree
# !pip install geopandas
import pandas as pd
import geopandas as gpd
import rtree
from shapely.geometry import Point, Polygon
root_path = '/content/drive/'
shapedata_path = root_path + 'MyDrive/Distributionpaper/ShapefileData/'
welldata_path = root_path + 'MyDrive/Distributionpaper/WellData/'
result_path = root_path + 'MyDrive/Distributionpaper/results/'

# Load state boundary shapefiles
states = gpd.GeoDataFrame.from_file(shapedata_path + 'cb_2018_us_state_500k.shp')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.4 MB/s eta 0:00:00


In [4]:
# csv to geodf
def cvt_point_to_geodf(df,latname='lat',lonname='lon'):
    df['coords'] = list(zip(df[lonname],df[latname]))
    df['coords'] = df['coords'].apply(Point)
    gdf = gpd.GeoDataFrame(df, geometry='coords')
    return gdf

# Add an indicator for whether a well was visited in a given month to a well-level data frame
def AssignMonthVisit(i,row):
    if row['month'] == i:
        return 1

# Some sort of weighting function, not sure what it does
def weight(m, basinname, basin_withcampaign_groupby):
  try:
    count_m = basin_withcampaign_groupby[(basin_withcampaign_groupby['Name'] == basinname) & (basin_withcampaign_groupby['month'] == m)].iloc[0]['count']
    count_t = np.sum(basin_withcampaign_groupby['count'])
    return count_m/count_t
  except IndexError:
    return 0

def weightByMonthlyVisits(m, monthly_visits):
  try:
    count_m = monthly_visits.iloc[m]
    count_t = monthly_visits.sum()
    return count_m/count_t
  except IndexError:
    return 0

def campaignmonthOilProd(m, w_in_campaign_groupby):
  return np.sum(w_in_campaign_groupby['Oil_' + m])

def basinmonthOilProd(m, basinname, w_in_basin_groupby):
  return w_in_basin_groupby[(w_in_basin_groupby['Name'] == basinname)].iloc[0]['Oil_' + m]

# Daily oil production in a given month by basin, weighted across the campaign
def basinmonthOilProdDaily(m, basinname, w_in_basin_groupby):
  d = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
  daysPerMonth = pd.DataFrame(data=d, index=[0]) #.transpose()
  return w_in_basin_groupby[(w_in_basin_groupby['Name'] == selectBasin)].iloc[0]['Oil_' + m]/daysPerMonth.iloc[0,int(m)-1]

def fractionOil(basinname, basin_withcampaign_groupby, w_in_basin_groupby, w_in_campaign_groupby):
  numerator = campaignmonthOilProd('01', w_in_campaign_groupby)*weight('01', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('02', w_in_campaign_groupby)*weight('02', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('03', w_in_campaign_groupby)*weight('03', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('04', w_in_campaign_groupby)*weight('04', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('05', w_in_campaign_groupby)*weight('05', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('06', w_in_campaign_groupby)*weight('06', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('07', w_in_campaign_groupby)*weight('07', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('08', w_in_campaign_groupby)*weight('08', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('09', w_in_campaign_groupby)*weight('09', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('10', w_in_campaign_groupby)*weight('10', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('11', w_in_campaign_groupby)*weight('11', basinname, basin_withcampaign_groupby) + campaignmonthOilProd('12', w_in_campaign_groupby)*weight('12', basinname, basin_withcampaign_groupby)
  denominator = basinmonthOilProd('01', basinname, w_in_basin_groupby)*weight('01', basinname, basin_withcampaign_groupby) + basinmonthOilProd('02', basinname, w_in_basin_groupby)*weight('02', basinname, basin_withcampaign_groupby) + basinmonthOilProd('03', basinname, w_in_basin_groupby)*weight('03', basinname, basin_withcampaign_groupby) + basinmonthOilProd('04', basinname, w_in_basin_groupby)*weight('04', basinname, basin_withcampaign_groupby) + basinmonthOilProd('05', basinname, w_in_basin_groupby)*weight('05', basinname, basin_withcampaign_groupby) + basinmonthOilProd('06', basinname, w_in_basin_groupby)*weight('06', basinname, basin_withcampaign_groupby) + basinmonthOilProd('07', basinname, w_in_basin_groupby)*weight('07', basinname, basin_withcampaign_groupby) + basinmonthOilProd('08', basinname, w_in_basin_groupby)*weight('08', basinname, basin_withcampaign_groupby) + basinmonthOilProd('09', basinname, w_in_basin_groupby)*weight('09', basinname, basin_withcampaign_groupby) + basinmonthOilProd('10', basinname, w_in_basin_groupby)*weight('10', basinname, basin_withcampaign_groupby) + basinmonthOilProd('11', basinname, w_in_basin_groupby)*weight('11', basinname, basin_withcampaign_groupby) + basinmonthOilProd('12', basinname, w_in_basin_groupby)*weight('12', basinname, basin_withcampaign_groupby)
  return numerator/denominator, numerator, denominator

# Daily gas production in a given campaign in a given month
def campaignmonthGasProd(m, w_in_campaign_groupby):
  return np.sum(w_in_campaign_groupby['Gas_' + m])

# Daily gas production in a given campaign in a given month
def basinmonthGasProd(m, basinname, w_in_basin_groupby):
  return w_in_basin_groupby[(w_in_basin_groupby['Name'] == basinname)].iloc[0]['Gas_' + m]

# Daily oil production in a given month by basin, weighted across the campaign
def basinmonthGasProdDaily(m, basinname, w_in_basin_groupby):
  d = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
  daysPerMonth = pd.DataFrame(data=d, index=[0]) #.transpose()
  return w_in_basin_groupby[(w_in_basin_groupby['Name'] == basinname)].iloc[0]['Gas_' + m]/daysPerMonth.iloc[0,int(m)-1]

# Compute average daily oil and gas production for each well in a data frame (from monthly input data)
# Monthly production data make it easy to do a properly weighted average (just divide total production over all months by days per year)
def generateAvgOilGasProd(wells):
  daysPerYear = 365
  wells['AvgDailyOil'] = (wells['Oil_01'] + wells['Oil_02'] + wells['Oil_03'] + wells['Oil_04'] + wells['Oil_05'] + wells['Oil_06'] + wells['Oil_07'] + wells['Oil_08'] + wells['Oil_09'] + wells['Oil_10'] + wells['Oil_11'] + wells['Oil_12'])/daysPerYear
  wells['AvgDailyGas'] = (wells['Gas_01'] + wells['Gas_02'] + wells['Gas_03'] + wells['Gas_04'] + wells['Gas_05'] + wells['Gas_06'] + wells['Gas_07'] + wells['Gas_08'] + wells['Gas_09'] + wells['Gas_10'] + wells['Gas_11'] + wells['Gas_12'])/daysPerYear
  return wells

def fractionGas(basinname, basin_withcampaign_groupby, w_in_basin_groupby, w_in_campaign_groupby):
  numerator = campaignmonthGasProd('01', w_in_campaign_groupby)*weight('01', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('02', w_in_campaign_groupby)*weight('02', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('03', w_in_campaign_groupby)*weight('03', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('04', w_in_campaign_groupby)*weight('04', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('05', w_in_campaign_groupby)*weight('05', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('06', w_in_campaign_groupby)*weight('06', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('07', w_in_campaign_groupby)*weight('07', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('08', w_in_campaign_groupby)*weight('08', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('09', w_in_campaign_groupby)*weight('09', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('10', w_in_campaign_groupby)*weight('10', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('11', w_in_campaign_groupby)*weight('11', basinname, basin_withcampaign_groupby) + campaignmonthGasProd('12', w_in_campaign_groupby)*weight('12', basinname, basin_withcampaign_groupby)
  denominator = basinmonthGasProd('01', basinname, w_in_basin_groupby)*weight('01', basinname, basin_withcampaign_groupby) + basinmonthGasProd('02', basinname, w_in_basin_groupby)*weight('02', basinname, basin_withcampaign_groupby) + basinmonthGasProd('03', basinname, w_in_basin_groupby)*weight('03', basinname, basin_withcampaign_groupby) + basinmonthGasProd('04', basinname, w_in_basin_groupby)*weight('04', basinname, basin_withcampaign_groupby) + basinmonthGasProd('05', basinname, w_in_basin_groupby)*weight('05', basinname, basin_withcampaign_groupby) + basinmonthGasProd('06', basinname, w_in_basin_groupby)*weight('06', basinname, basin_withcampaign_groupby) + basinmonthGasProd('07', basinname, w_in_basin_groupby)*weight('07', basinname, basin_withcampaign_groupby) + basinmonthGasProd('08', basinname, w_in_basin_groupby)*weight('08', basinname, basin_withcampaign_groupby) + basinmonthGasProd('09', basinname, w_in_basin_groupby)*weight('09', basinname, basin_withcampaign_groupby) + basinmonthGasProd('10', basinname, w_in_basin_groupby)*weight('10', basinname, basin_withcampaign_groupby) + basinmonthGasProd('11', basinname, w_in_basin_groupby)*weight('11', basinname, basin_withcampaign_groupby) + basinmonthGasProd('12', basinname, w_in_basin_groupby)*weight('12', basinname, basin_withcampaign_groupby)
  return numerator/denominator, numerator, denominator

monthStr = pd.DataFrame(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
monthNum = pd.DataFrame(np.arange(12))

In [42]:
# Generate survey well visit and production coverage summary results for the Kairos Permian campaign

wellvisits2018 = pd.read_csv(welldata_path + 'Stanford-PermianNM-ProductionRequest-2018Final.csv')
wellvisits2019 = pd.read_csv(welldata_path + 'Stanford-PermianNM-ProductionRequest-2019Final.csv')
wellvisits2020 = pd.read_csv(welldata_path + 'Stanford-PermianNM-ProductionRequest-2020Final.csv')

# Update oil, gas, and visit column names
originalCols = wellvisits2018.columns
wellvisits2018.columns = '18' + wellvisits2018.columns
wellvisits2019.columns = '19' + wellvisits2019.columns
wellvisits2020.columns = '20' + wellvisits2020.columns

# Create unified data frame
wellvisits2018_2020 = wellvisits2018.join(wellvisits2019).join(wellvisits2020)
wellvisits2018_2020.rename(columns={'18AssetID':'AssetID', '18asset_type':'asset_type'}, inplace=True)

# Generate final well columns
# month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_num_nozero = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
oil_cols_general = ['Oil_' + i for i in month_num_nozero]
gas_cols_general = ['Gas_' + i for i in month_num_nozero]
visit_cols_general = ['Visit_' + i for i in month_num_nozero]
oil_cols_18 = ['18' + i for i in oil_cols_general]
oil_cols_19 = ['19' + i for i in oil_cols_general]
oil_cols_20 = ['20' + i for i in oil_cols_general]
oil_cols_all = oil_cols_18 + oil_cols_19 + oil_cols_20
gas_cols_18 = ['18' + i for i in gas_cols_general]
gas_cols_19 = ['19' + i for i in gas_cols_general]
gas_cols_20 = ['20' + i for i in gas_cols_general]
gas_cols_all = gas_cols_18 + gas_cols_19 + gas_cols_20
visit_cols_18 = ['18' + i for i in visit_cols_general]
visit_cols_19 = ['19' + i for i in visit_cols_general]
visit_cols_20 = ['20' + i for i in visit_cols_general]
visit_cols_all = visit_cols_18 + visit_cols_19 + visit_cols_20
final_wells_columns = ['AssetID'] + gas_cols_all + oil_cols_all + visit_cols_all
wellvisits2018_2020 = wellvisits2018_2020.loc[:, final_wells_columns]
# Drop wells with no production
wellvisits2018_2020['total_oil'] = wellvisits2018_2020.loc[:, oil_cols_all].sum(axis=1)
wellvisits2018_2020['total_gas'] = wellvisits2018_2020.loc[:, gas_cols_all].sum(axis=1)
wellvisits2018_2020 = wellvisits2018_2020[(wellvisits2018_2020['total_oil'] + wellvisits2018_2020['total_gas']) > 0]

d = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
d_list = list(d.values())
d_list_all_years = d_list + d_list + d_list
# Note that no data in this dataset were collected in the leap month of February 2022
daysPerMonthAll = pd.DataFrame(pd.DataFrame(d_list_all_years).transpose())

daysPerMonthOil = daysPerMonthAll.copy()
daysPerMonthOil.columns = oil_cols_all
daysPerMonthOil = daysPerMonthOil.loc[0,:]

daysPerMonthGas = daysPerMonthAll.copy()
daysPerMonthGas.columns = gas_cols_all
daysPerMonthGas = daysPerMonthGas.loc[0,:]

daysPerMonthVisit = daysPerMonthAll.copy()
daysPerMonthVisit.columns = visit_cols_all
daysPerMonthVisit = daysPerMonthVisit.loc[0,:]

# Count production covered on a repeated basis (if a given well is visited twice, its production is counted twice)
# Normalize from monthly to daily production (none of the campaigns cover February, so no need to worry about leap days)
# mcf(NG)/d

# if (campaignname == 'Null'):
#     slct_basin_campaign_summary_stats = pd.DataFrame(['Null'])
# else:
wellvisits2018_2020['rpt_Oil'] = (wellvisits2018_2020[oil_cols_all] * np.array(wellvisits2018_2020[visit_cols_all]) / daysPerMonthOil).sum(axis=1)
wellvisits2018_2020['rpt_Gas'] = (wellvisits2018_2020[gas_cols_all] * np.array(wellvisits2018_2020[visit_cols_all]) / daysPerMonthGas).sum(axis=1)

# Computes average daily oil and natural gas production volume for each covered well (weighting for the length of each month)
wellvisits2018_2020['avg_Oil'] = (wellvisits2018_2020[oil_cols_all] * np.array(wellvisits2018_2020[visit_cols_all])).sum(axis=1) / np.array(wellvisits2018_2020[visit_cols_all]*daysPerMonthVisit).sum(axis=1)
wellvisits2018_2020['avg_Gas'] = (wellvisits2018_2020[gas_cols_all] * np.array(wellvisits2018_2020[visit_cols_all])).sum(axis=1) / np.array(wellvisits2018_2020[visit_cols_all]*daysPerMonthVisit).sum(axis=1)

# Add total visit count for each well
wellvisits2018_2020['rpt_well_count'] = np.array(wellvisits2018_2020[visit_cols_all]).sum(axis=1)

# Drop non-producing wells
# wellvisits2018_2020 = wellvisits2018_2020[wellvisits2018_2020['well_status'] =

# Sort by well API number
wellvisits2018_2020 = wellvisits2018_2020.sort_values(by=['AssetID'])

# Add dummy columns for aggregation and counting productive wells
wellvisits2018_2020['Prov_cod_1'] = 1
wellvisits2018_2020['well_count'] = 1

# Aggregate well-level summary statistics by basin
wellvisits2018_2020_groupby = wellvisits2018_2020.groupby(['Prov_cod_1']).agg({'AssetID': 'first', 'well_count':np.sum, 'rpt_well_count': np.sum, 'avg_Oil': np.sum, 'rpt_Oil': np.sum, 'avg_Gas': np.sum, 'rpt_Gas': np.sum})
wellvisits2018_2020_groupby = wellvisits2018_2020_groupby.reset_index()
wellvisits2018_2020_groupby.head()

# Convert from monthly to daily production numbers
d = {'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31}
d_list = list(d.values())
d_list_all_years = d_list + d_list + d_list
# Note that no data in this dataset were collected in the leap month of February 2022
daysPerMonthAll = pd.DataFrame(pd.DataFrame(d_list_all_years).transpose())

daysPerMonthOil = daysPerMonthAll.copy()
daysPerMonthOil.columns = oil_cols_all
daysPerMonthOil = daysPerMonthOil.loc[0,:]

daysPerMonthGas = daysPerMonthAll.copy()
daysPerMonthGas.columns = gas_cols_all
daysPerMonthGas = daysPerMonthGas.loc[0,:]

daysPerMonthVisit = daysPerMonthAll.copy()
daysPerMonthVisit.columns = visit_cols_all
daysPerMonthVisit = daysPerMonthVisit.loc[0,:]

# Count production covered on a repeated basis (if a given well is visited twice, its production is counted twice)
# Normalize from monthly to daily production (none of the campaigns cover February, so no need to worry about leap days)
# mcf(NG)/d

# if (campaignname == 'Null'):
#     slct_basin_campaign_summary_stats = pd.DataFrame(['Null'])
# else:
wellvisits2018_2020['rpt_Oil'] = (wellvisits2018_2020[oil_cols_all] * np.array(wellvisits2018_2020[visit_cols_all]) / daysPerMonthOil).sum(axis=1)
wellvisits2018_2020['rpt_Gas'] = (wellvisits2018_2020[gas_cols_all] * np.array(wellvisits2018_2020[visit_cols_all]) / daysPerMonthGas).sum(axis=1)

# Computes average daily oil and natural gas production volume for each covered well (weighting for the length of each month)
wellvisits2018_2020['avg_Oil'] = (wellvisits2018_2020[oil_cols_all] * np.array(wellvisits2018_2020[visit_cols_all])).sum(axis=1) / np.array(wellvisits2018_2020[visit_cols_all]*daysPerMonthVisit).sum(axis=1)
wellvisits2018_2020['avg_Gas'] = (wellvisits2018_2020[gas_cols_all] * np.array(wellvisits2018_2020[visit_cols_all])).sum(axis=1) / np.array(wellvisits2018_2020[visit_cols_all]*daysPerMonthVisit).sum(axis=1)

# Add total visit count for each well
wellvisits2018_2020['rpt_well_count'] = np.array(wellvisits2018_2020[visit_cols_all]).sum(axis=1)

# Drop non-producing wells
# wellvisits2018_2020 = wellvisits2018_2020[wellvisits2018_2020['well_status'] =

# Sort by well API number
wellvisits2018_2020 = wellvisits2018_2020.sort_values(by=['AssetID'])

# Add dummy columns for aggregation and counting productive wells
wellvisits2018_2020['Prov_cod_1'] = 1
wellvisits2018_2020['well_count'] = 1

# Aggregate well-level summary statistics by basin
wellvisits2018_2020_groupby = wellvisits2018_2020.groupby(['Prov_cod_1']).agg({'AssetID': 'first', 'well_count':np.sum, 'rpt_well_count': np.sum, 'avg_Oil': np.sum, 'rpt_Oil': np.sum, 'avg_Gas': np.sum, 'rpt_Gas': np.sum})
wellvisits2018_2020_groupby = wellvisits2018_2020_groupby.reset_index()

# Import well data for the whole New Mexico Permian
wells2018 = pd.read_csv(welldata_path + 'NM_Permian_production_2018.csv')
wells2019 = pd.read_csv(welldata_path + 'NM_Permian_production_2019.csv')
wells2020 = pd.read_csv(welldata_path + 'NM_Permian_production_2020.csv')

# Add daily average oil and gas production columns
wells2018 = generateAvgOilGasProd(wells2018)
wells2019 = generateAvgOilGasProd(wells2019)
wells2020 = generateAvgOilGasProd(wells2020)

# Update oil, gas, and visit column names
originalCols = wells2018.columns
wells2018.columns = '18' + wells2018.columns
wells2019.columns = '19' + wells2019.columns
wells2020.columns = '20' + wells2020.columns

# Create unified data frame
wells2018_2020 = wells2018.join(wells2019).join(wells2020)
wells2018_2020.rename(columns={'18Asset ID':'Asset ID', '18asset_type':'asset_type', '18production_type':'production_type', '18well_status':'well_status'}, inplace=True)

# Generate final well columns
month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_num_nozero = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
oil_cols_general_basin = ['Oil_' + i for i in month_num]
gas_cols_general_basin = ['Gas_' + i for i in month_num]
visit_cols_general_basin = ['Visit_' + i for i in month_num_nozero]
oil_cols_18_basin = ['18' + i for i in oil_cols_general_basin]
oil_cols_19_basin = ['19' + i for i in oil_cols_general_basin]
oil_cols_20_basin = ['20' + i for i in oil_cols_general_basin]
oil_cols_all_basin = oil_cols_18_basin + oil_cols_19_basin + oil_cols_20_basin
gas_cols_18_basin = ['18' + i for i in gas_cols_general_basin]
gas_cols_19_basin = ['19' + i for i in gas_cols_general_basin]
gas_cols_20_basin = ['20' + i for i in gas_cols_general_basin]
gas_cols_all_basin = gas_cols_18_basin + gas_cols_19_basin + gas_cols_20_basin
visit_cols_18_basin = ['18' + i for i in visit_cols_general_basin]
visit_cols_19_basin = ['19' + i for i in visit_cols_general_basin]
visit_cols_20_basin = ['20' + i for i in visit_cols_general_basin]
visit_cols_all_basin = visit_cols_18_basin + visit_cols_19_basin + visit_cols_20_basin
final_wells_columns_basin = ['Asset ID'] + gas_cols_all_basin + oil_cols_all_basin + visit_cols_all_basin # + ['asset_type', 'production_type', 'well_status']
wells2018_2020 = wells2018_2020.loc[:, final_wells_columns_basin]

# Switch to the column naming convention from the Kairos data with visits
wells2018_2020.columns = final_wells_columns

# Convert from monthly to daily production
wells2018_2020.loc[:, gas_cols_all] = wells2018_2020.loc[:, gas_cols_all]/daysPerMonthGas
wells2018_2020.loc[:, oil_cols_all] = wells2018_2020.loc[:, oil_cols_all]/daysPerMonthOil

# Drop non-producing wells
wells2018_2020['total_gas'] = (wells2018_2020.loc[:, gas_cols_all]).sum(axis=1)
wells2018_2020['total_oil'] = (wells2018_2020.loc[:, oil_cols_all]).sum(axis=1)

# wellvisits2018_2020['total_oil'] = wellvisits2018_2020.loc[:, oil_cols_all].sum(axis=1)
# wellvisits2018_2020['total_gas'] = wellvisits2018_2020.loc[:, gas_cols_all].sum(axis=1)
# wells2018_2020 = wells2018_2020[(wells2018_2020['production_type']=='OIL') & (wells2018_2020['production_type']=='GAS')]
# wells2018_2020 = wells2018_2020.loc[(wells2018_2020['total_oil'] > 0) * (wells2018_2020['total_gas'] > 0)]
wells2018_2020 = wells2018_2020.loc[((wells2018_2020['total_oil'] > 0) + (wells2018_2020['total_gas'] > 0)) > 0]

# Well visits by month
basin_gas_prod_2018_2020 = wells2018_2020.loc[:, gas_cols_all].sum(axis=0)
basin_oil_prod_2018_2020 = wells2018_2020.loc[:, oil_cols_all].sum(axis=0)
wellvisits_bymonth = wellvisits2018_2020.loc[:, visit_cols_all].sum(axis=0)

NM_basin_prod_visits_2018_2020 = pd.DataFrame({'GasProd':basin_gas_prod_2018_2020.values, 'OilProd':basin_oil_prod_2018_2020.values, 'VisitCount':wellvisits_bymonth.values})
# basin_avg_prod_campaign = NM_basin_prod_visits_2018_2020*NM_basin_prod_visits_2018_2020['VisitCount']/NM_basin_prod_visits_2018_2020['VisitCount'].sum(axis=0)
# basin_avg_prod_campaign

# Compute visit-weighted average basin-wide production
totVisits = NM_basin_prod_visits_2018_2020['VisitCount'].sum(axis=0)
NM_basin_prod_visits_2018_2020['GasProdxVisitCount'] = NM_basin_prod_visits_2018_2020['GasProd']*NM_basin_prod_visits_2018_2020['VisitCount']/totVisits
NM_basin_prod_visits_2018_2020['OilProdxVisitCount'] = NM_basin_prod_visits_2018_2020['OilProd']*NM_basin_prod_visits_2018_2020['VisitCount']/totVisits
basin_summary = pd.DataFrame(NM_basin_prod_visits_2018_2020.agg({'OilProdxVisitCount':np.sum, 'GasProdxVisitCount':np.sum, 'VisitCount':np.sum})).transpose()
basin_summary['WellCount'] = wells2018_2020.shape[0]
wellvisits2018_2020_groupby['Prov_cod_1'] = '430'
wellvisits2018_2020_groupby['basin_well_count'] = basin_summary['WellCount']
wellvisits2018_2020_groupby['Total basin oil production'] = basin_summary['OilProdxVisitCount']
wellvisits2018_2020_groupby['Total basin gas production'] = basin_summary['GasProdxVisitCount']
wellvisits2018_2020_groupby['Fraction oil production covered'] = wellvisits2018_2020_groupby['avg_Oil']/wellvisits2018_2020_groupby['Total basin oil production']
wellvisits2018_2020_groupby['Fraction gas production covered'] = wellvisits2018_2020_groupby['avg_Gas']/wellvisits2018_2020_groupby['Total basin gas production']
wellvisits2018_2020_groupby['Basin well productivity'] = wellvisits2018_2020_groupby['avg_Gas']/wellvisits2018_2020_groupby['basin_well_count']
wellvisits2018_2020_groupby['Covered well productivity'] = wellvisits2018_2020_groupby['avg_Gas']/wellvisits2018_2020_groupby['well_count']
wellvisits2018_2020_groupby['Covered well productivity with repeats'] = wellvisits2018_2020_groupby['rpt_Gas']/wellvisits2018_2020_groupby['rpt_well_count']
wellvisits2018_2020_groupby

,Prov_cod_1,AssetID,well_count,rpt_well_count,avg_Oil,rpt_Oil,avg_Gas,rpt_Gas,basin_well_count,Total basin oil production,Total basin gas production,Fraction oil production covered,Fraction gas production covered,Basin well productivity,Covered well productivity,Covered well productivity with repeats
0,430,0,21637,81564.0,732226.357636,3.192910e+06,2.703944e+06,1.258937e+07,28429,763906.529508,2.904806e+06,0.958529,0.930852,95.112161,124.968508,154.349538


In [41]:
# wellvisits2018_2020.columns
# ((wellvisits2018_2020['total_gas'] + wellvisits2018_2020['total_oil'])).shape
# len(list(set().union(wells2018['18Asset ID'], wells2019['19Asset ID'], wells2020['20Asset ID'])))
# wells2018.head()
# len(wells2020['20Asset ID'])
# Import well data for the whole New Mexico Permian
wells2018 = pd.read_csv(welldata_path + 'NM_Permian_production_2018.csv')
wells2019 = pd.read_csv(welldata_path + 'NM_Permian_production_2019.csv')
wells2020 = pd.read_csv(welldata_path + 'NM_Permian_production_2020.csv')

# Add daily average oil and gas production columns
wells2018 = generateAvgOilGasProd(wells2018)
wells2019 = generateAvgOilGasProd(wells2019)
wells2020 = generateAvgOilGasProd(wells2020)

# Update oil, gas, and visit column names
originalCols = wells2018.columns
wells2018.columns = '18' + wells2018.columns
wells2019.columns = '19' + wells2019.columns
wells2020.columns = '20' + wells2020.columns

# Create unified data frame
wells2018_2020 = wells2018.join(wells2019).join(wells2020)
wells2018_2020.rename(columns={'18Asset ID':'Asset ID', '18asset_type':'asset_type', '18production_type':'production_type', '18well_status':'well_status'}, inplace=True)

# Generate final well columns
month_num = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
month_num_nozero = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
oil_cols_general_basin = ['Oil_' + i for i in month_num]
gas_cols_general_basin = ['Gas_' + i for i in month_num]
visit_cols_general_basin = ['Visit_' + i for i in month_num_nozero]
oil_cols_18_basin = ['18' + i for i in oil_cols_general_basin]
oil_cols_19_basin = ['19' + i for i in oil_cols_general_basin]
oil_cols_20_basin = ['20' + i for i in oil_cols_general_basin]
oil_cols_all_basin = oil_cols_18_basin + oil_cols_19_basin + oil_cols_20_basin
gas_cols_18_basin = ['18' + i for i in gas_cols_general_basin]
gas_cols_19_basin = ['19' + i for i in gas_cols_general_basin]
gas_cols_20_basin = ['20' + i for i in gas_cols_general_basin]
gas_cols_all_basin = gas_cols_18_basin + gas_cols_19_basin + gas_cols_20_basin
visit_cols_18_basin = ['18' + i for i in visit_cols_general_basin]
visit_cols_19_basin = ['19' + i for i in visit_cols_general_basin]
visit_cols_20_basin = ['20' + i for i in visit_cols_general_basin]
visit_cols_all_basin = visit_cols_18_basin + visit_cols_19_basin + visit_cols_20_basin
final_wells_columns_basin = ['Asset ID'] + gas_cols_all_basin + oil_cols_all_basin + visit_cols_all_basin # + ['asset_type', 'production_type', 'well_status']
wells2018_2020 = wells2018_2020.loc[:, final_wells_columns_basin]

# Switch to the column naming convention from the Kairos data with visits
wells2018_2020.columns = final_wells_columns

# Convert from monthly to daily production
wells2018_2020.loc[:, gas_cols_all] = wells2018_2020.loc[:, gas_cols_all]/daysPerMonthGas
wells2018_2020.loc[:, oil_cols_all] = wells2018_2020.loc[:, oil_cols_all]/daysPerMonthOil

# Drop non-producing wells
wells2018_2020['total_gas'] = (wells2018_2020.loc[:, gas_cols_all]).sum(axis=1)
wells2018_2020['total_oil'] = (wells2018_2020.loc[:, oil_cols_all]).sum(axis=1)

wells2018_2020 = wells2018_2020.loc[((wells2018_2020['total_oil'] > 0) + (wells2018_2020['total_gas'] > 0)) > 0]
wells2018_2020.shape

(28429, 111)

In [ ]:
# Save well-level gas productivity quantiles
todaydate = '20221021'
quantiles = np.arange(0, 1.001, 0.001)
covered_gas_prod_quantiles_by_well = wellvisits2018_2020['avg_Gas'].quantile(quantiles)
covered_gas_prod_quantiles_by_well.to_csv(result_path + 'Covered_gas_prod_percentiles_all_basins_by_well_Kairos_NM_Permian' + todaydate + '.csv', index=False)